In [6]:
import os

# Define constants
INPUT_DIR = "data/input"
OUTPUT_DIR = "data/output"
INSTITUTION = "Zerodha"

# Ensure output dir exists
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [11]:
# load common functions
from importlib.machinery import SourceFileLoader

common_lib = SourceFileLoader("common_lib", "../common_lib/common_functions.py").load_module()

from common_lib import *

In [12]:
def get_balances(df, holding_type):
    date_str = df[holding_type].iloc[9][1].split(" Holdings Statement as on ")[-1]
    return (date_str, df[holding_type].iloc[13][2], df[holding_type].iloc[14][2])


In [13]:
import pandas as pd
from datetime import datetime

def get_holdings(filepath):
    df = pd.read_excel(filepath, sheet_name=None)
    output = {}
    for holding_type in ["Equity", "Mutual Funds"]:
        date_str, invested, present = get_balances(df, holding_type)
        statement_date = datetime.fromisoformat(date_str).date()
        statement_date = get_last_date_of_month(statement_date)
        output[holding_type] = {"date": statement_date, "invested": invested, "present": present}
    
    return output


In [14]:
def generate_monthly_balances():
    all_files = get_all_excel_paths(INPUT_DIR)
    all_files.sort(reverse=True)
    output = ["Date,Institution,Account Type,Present Value INR,Invested Value,Present Value USD,Comments"]

    for filepath in all_files:
        file_output = []
        invalid_file = False

        for holding_type, holdings in get_holdings(filepath).items():
            if holdings["invested"] == 0 or holdings["present"] == 0:
                print("Invalid data for date: ", holdings["date"])
                invalid_file = True
                break

            # rate = er.get_inr_rate(holdings["date"])
            # file_output.append(f"{holdings["date"].isoformat()},{INSTITUTION},{holding_type},{holdings["invested"]},{holdings["present"]},{(holdings["present"] / rate):.2f},")
            file_output.append(f"{holdings['date'].isoformat()},{INSTITUTION},{holding_type},{holdings['present']:.2f},{holdings['invested']:.2f},,")

        if not invalid_file:
            output += file_output

    return output


In [ ]:
output = generate_monthly_balances()
with open(f"{OUTPUT_DIR}/{INSTITUTION}-Monthly-Balances.csv", "w") as f:
    for line in output:
        f.write(f"{line}\n")